# Imports

In [6]:
import yfinance as yf 
import pandas as pd
import sqlite3
from configparser import ConfigParser
import datetime
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import BusinessDay
import holidays
import matplotlib
import logging

# 1- Yahoo data retrieval

In [44]:
def oil_futures_ticker_generator(date):
    """ Builds a ticker for WTI contract expiring in that month/year combination.
    
    Uses a dictionary to get the mapping of months to ticker codes and builds a ticker code.

    Parameters
    ----------
    date : datetime.date
        A datetime.date object of that day/month/year of the desired ticker

    Returns
    ----------
    ticker : string
        The string of the ticker that will be used by the data interface
    """
    #Map months to ticker codes
    month_symbol_mapper = {
    '1':{'string':'jan','code':'F'},
    '2':{'string':'feb','code':'G'},
    '3':{'string':'mar','code':'H'},
    '4':{'string':'apr','code':'J'},
    '5':{'string':'may','code':'K'},
    '6':{'string':'jun','code':'M'},
    '7':{'string':'jul','code':'N'},
    '8':{'string':'aug','code':'Q'},
    '9':{'string':'sep','code':'U'},
    '10':{'string':'oct','code':'V'},
    '11':{'string':'nov','code':'X'},
    '12':{'string':'dev','code':'Z'},
    }

    ticker = f"CL{month_symbol_mapper[str(date.month)]['code']}{date.year-2000}.NYM"
    return ticker


def get_business_date_offset(date):
    """ Get the business date offset WTI oil futures using the specs from ICE.
    
    The contracts settle 3 business days prior to the 25th of each month. If the 25th is not a business date itself
    the offset is 4 business days.

    Parameters
    ----------
    date : datetime.date
        The 25th day of the month in which we're looking for an expiry date

    Returns
    ----------
    offset : int
        The number of days to offset from the 25th day of the month
    """

    #builds a dictionary of US holidays
    us_holidays = holidays.US()

    offset = 3 if date.weekday() < 5 and date not in us_holidays else 4
    return offset


def get_25th_day(date):
    """ Get the 25th day of a given month.
    
    The 25th day of a month is the anchor from which ICE determines the offset for contract expiry. If 25th is a weekday
    then the offset is 3 business days otherwise it is 4. 

    Parameters
    ----------
    date : datetime.date
        The 25th day of the month in which we're looking for an expiry date

    Returns
    ----------
    offset : int
        The number of days to offset from the 25th day of the month
    """

    assert  isinstance(date, datetime.date), 'date passed is not a datetime object'
    return datetime.date(date.year, date.month, 25)


def get_contract_code_time_spread(date=datetime.date.today(), time_spread=0):
    """ Get the next settlement contract from a given date.
    
    Contracts usualy expire on 20-22nd day of a given month prior to contract delivery. Depending on which day it is the
    contract expiry can be determined for this month or any given number of months into the future. To get the current 
    (next) expiry, use the default values. Otherwise use the next contract expiry as an anchor and the time_spread steps
    as the offset for months into the future.

    Parameters
    ----------
    date : datetime.date
        The date from which you want to get the expiry dates.
    
    time_spread : int
        the number of months into the future that you want to ge the expiry for.

    Returns
    ----------
    ticker : string
        The ticker for the returned contract.

    next_settelment_date : datetime.date
        The settlment date for the returned contract.
    """

    #get the 25th day of the month, the anchor
    day_25th = get_25th_day(date) + relativedelta(months=time_spread)
    
    #get the offset depending the which weekday the 25th is on
    business_day_offset = get_business_date_offset(day_25th)

    #The settlement date logic
    settlement_date_current_month = day_25th -  business_day_offset*BusinessDay()
    settlement_date_current_month = settlement_date_current_month.to_pydatetime().date()
    
    #If already passed the settlment date for the month, then use the next months
    if date < settlement_date_current_month:
        next_settelment_date = settlement_date_current_month
    else:
        day_25th = day_25th + relativedelta(months=1)
        business_day_offset = get_business_date_offset(day_25th)
        next_settelment_date = day_25th - business_day_offset*BusinessDay()
        next_settelment_date = next_settelment_date.to_pydatetime().date()

    #Ticker is generated using the expiry date
    ticker = oil_futures_ticker_generator(next_settelment_date+relativedelta(months=1))

    logging.debug(f"2 next_settelment_date : {next_settelment_date}, ticker: {ticker}")
    
    return ticker, next_settelment_date


def create_databse_connection(db_file='None'):
        """ Creates a database connection to the SQLite database.

        Given a database file name, usualy from a config file, creates an sqlite3 connection object.

        Parameters
        ----------
        db_file : string
            The name of the database file.

        Returns
        ----------
        conn : sqlite3_connect
            The sqlite3 connection onbject
        
        cur : sqlite3_connect.cursor
            The sqlite3 connection cursor
    
        """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
            cur = conn.cursor()
            return conn, cur
        except Exception as e:
            logging.error(f"Problem with connecting to database {db_file}, {e}")


def get_latest_date(database, table, ticker=None):
    """ Get the max date available in a given table for a given ticker.

    When looking for a date on which to start your data retrieval from a service such as yahoo, you wouldn't want to
    be requesting data that is already in your database. This would make your requests and processing run for longer
    uneccessarly. If the database doesn't currently have data on that ticker then we pull the data starting 45 days
    ago.

    Parameters
    ----------
    database : string
        Database that has the appropriate data
    
    table : string
        Table from which you want to retrieve the date
    
    ticker : string
        Ticker for which you need the latest date available
    
    Returns
    ----------
    latest_date : datetime.datetime
        Latest date available in the database
    """
    #setup connection
    conn, cur = create_databse_connection(db_file=database)
    
    #execute query
    if ticker is None:
        cur.execute(f"SELECT MAX(DATE) FROM '{table}';")
    else:
        cur.execute(f"SELECT MAX(DATE) FROM '{table}' WHERE TICKER = '{ticker}';")
    
    #parse data within the cursor
    res = cur.fetchone()
    conn.close()
    if res[0] is not None :
        latest_date = datetime.datetime.strptime(res[0], '%Y-%m-%d')
    else:
        latest_date = datetime.datetime.now() - datetime.timedelta(45)
    logging.debug(f"Start date for fetching yahoo data is {latest_date}")

    return latest_date.date()


class yahoo_futures_interface:
    """ Yahoo finance interface to retrieve data on futures contracts.

    Parameters
    ----------
    ticker : string
        Ticker in Yahoo Finance for the ticker of interest.
    
    contract_expiration : datetime.date
        Expiry date of the contract

    Methods
    ----------
    get_yahoo_data(self, start=None)
        Retrieves data from Yahoo Finance given a start date. Returns a dataframe that has been
        modified.
    
    create_entries_from_dataframe(self)
        creates and executes individual insert statements into the futures historival database.
    """

    def __init__(self, ticker, contract_expiration, start_date, database):
        self.ticker = ticker
        self.contract_expiration = contract_expiration
        self.start_date = start_date 
        self.database = database

    def get_yahoo_data(self, start=None):
        logging.info(f"Getting yahoo data for {self.ticker} from start date {self.start_date}")
        yahoo_ticker_interface = yf.Ticker(self.ticker)
        self.price_data = yahoo_ticker_interface.history(start=self.start_date, end=datetime.datetime.now())
        
        try:
            self.price_data.drop(labels=['Dividends', 'Stock Splits'], axis=1, inplace=True)
            self.price_data['Ticker'] = self.ticker
            self.price_data['Settlement Date'] = self.contract_expiration
            #remove from prod
            print(self.price_data.head()) 
        except Exception as e:
            logging.error(f"Facing error in getting data from yahoo: {e}")
    
    def create_entries_from_dataframe(self):
        for index,row in self.price_data.iterrows():
            logging.debug(f"Data to insert into the database: {index.date(),row[5],row[0],row[1],row[2],row[3],row[4],row[6]}")
            row = (index.date(),row[5],row[0],row[1],row[2],row[3],row[4],row[6])
            self._create_execute_entry(row)

    def _create_connection(self):
        """ create a database connection to the SQLite database specified by db_file"""
        self.conn = None
        try:
            self.conn = sqlite3.connect(self.database)
            self.cur = self.conn.cursor()
        except Exception as e:
            logging.error(f"Problem with connecting to database {self.database}, {e}")

    def _create_execute_entry(self, row):
        """Create a new futures price entry in the database's OIL_FUTURES_YAHOO table"""
        
        sql = '''INSERT INTO OIL_FUTURES_YAHOO(DATE,TICKER,OPEN, HIGH, LOW, CLOSE, VOLUME,SETTLEMENT_DATE)
                VALUES(?,?,?,?,?,?,?,?)'''
        try:
            self._create_connection()
            self.cur.execute(sql,row)
            self.conn.commit()
            self.conn.close()
        except Exception as e:
            logging.error(f"Error in inserting sql into the database: {e}")
         


## 1.1- recreate the create database entries from yahoo data workflow

In [56]:
today = datetime.date.today()
database = '../futures_dev.db'
table = 'OIL_FUTURES_YAHOO'
contracts = []
for step in range(0,24):
    if step==0:
        ticker, front_settelment_date = get_contract_code_time_spread()
        contracts.append((ticker, front_settelment_date))
    else:
        ticker, next_settelment_date = get_contract_code_time_spread(date = front_settelment_date, time_spread = step)
        contracts.append((ticker, next_settelment_date))


for contract in contracts[1:2]:
    ticker = contract[0]
    expiry = contract[1]
    latest_date = get_latest_date(database = database, table = table, ticker = ticker)
    print(f"The latest date available is {latest_date}")
    if latest_date < today:
        print(f"Latest date in futures data is {latest_date} and it's behind today's date, {today}")
        interface = yahoo_futures_interface(ticker = ticker, contract_expiration = expiry, start_date = latest_date + relativedelta(days=1), database = database)
        interface.get_yahoo_data()
        interface.create_entries_from_dataframe()
    else:
        print('data is up to date')
    
    


The latest date available is 2022-05-31
data is up to date


## 1.2- test out scenario where we try pulling data for a ticker that hasen't yet got any data in the database

In [53]:
latest_date = get_latest_date(database, table, ticker='CLQ22.NYM')
print(latest_date)

2022-05-31


In [66]:
import os 
from pathlib import Path
Path(os.path.dirname('/Users/karimitani/Documents/github/futures_data_manager'))

'/Users/karimitani/Documents/github'

In [69]:
Path(os.path.realpath('/Users/karimitani/Documents/github/futures_data_manager')).parent

PosixPath('/Users/karimitani/Documents/github')

In [5]:
import yfinance as yf
import datetime
yahoo_ticker_interface = yf.Ticker('CLJ24.NYM')
yahoo_ticker_interface.history(start= datetime.date(2022,5,27),end=datetime.datetime.now())


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-05-27,83.980003,83.980003,83.980003,83.980003,110,0,0
